In [ ]:
import os
import anthropic

from langchain_anthropic.chat_models import ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# Chat 모델 초기화 
chat_model = ChatAnthropic(
  model="claude-3-opus-20240229",
  temperature=0.1
)

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a cryptocurrency market expert. And you only reply in Korean."), 
  AIMessage(content="안녕하세요!"), 
  HumanMessage(content="Do you know Definalist?")
]

chat_model.predict_messages(messages)

/var/folders/ry/tfr9thnd55s7v76d7vr2pgrm0000gn/T/ipykernel_56324/889352447.py:9: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat_model.predict_messages(messages)


AIMessage(content='네, Definalist에 대해 알고 있습니다. Definalist는 블록체인 기술을 활용한 탈중앙화 금융(DeFi) 프로젝트 중 하나입니다. 이들은 사용자들이 가상자산을 예치하고 대출을 받을 수 있는 플랫폼을 제공하고 있죠. \n\nDefinalist의 특징은 다양한 암호화폐를 지원하며, 안정적이고 투명한 시스템을 갖추고 있다는 점입니다. 또한 사용자들에게 매력적인 수익률을 제공하는 것으로 알려져 있습니다.\n\n다만 DeFi 프로젝트는 아직 초기 단계이고 규제가 미비한 상황이라 리스크가 존재하는 것도 사실입니다. 투자에 앞서 프로젝트에 대한 충분한 이해와 리서치가 필요할 것 같네요. Definalist가 앞으로 어떻게 성장해 나갈지 지켜볼 만한 프로젝트라고 생각합니다.', additional_kwargs={}, response_metadata={'id': 'msg_01SbuRNvZAduZw4y2BhspLmm', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 51, 'output_tokens': 362}}, id='run-9d864597-d7d9-426e-980d-e8a84af32cb2-0', usage_metadata={'input_tokens': 51, 'output_tokens': 362, 'total_tokens': 413, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})